# Speed optimization dataframe

In [2]:
import os
import re
import json
import math
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

## General parameters

In [68]:
project_path = os.path.dirname(os.path.abspath('.'))
nso_path = f'{project_path}/output/solstorm/alns/managerial/speed_opt/nso/'
so_path = f'{project_path}/output/solstorm/alns/managerial/speed_opt/so/'

instance_key = 'instance'
nso_p_key = 'nso (p)'
so_p_key = 'so (p)'
nso_m_key = 'nso (m)'
so_m_key = 'so (m)'
nso_c_key = 'nso (c)'
so_c_key = 'so (c)'
red_fc_p_key = 'cost red (p)'
red_fc_m_key = 'cost red (m)'
red_fc_c_key = 'cost red (c)'
red_em_p_key = 'em red (p)'
red_em_m_key = 'em red (m)'
red_em_c_key = 'em red (c)'

perfect_key = 'perfect'
mixed_key = 'mixed'
critical_key = 'critical'

best_obj_key = 'best_objective'

## Functions

In [75]:
def generate_speed_opt_df(nso_path, so_path):
    nso_data = map_instance_to_ws_to_obj(nso_path)
    so_data = map_instance_to_ws_to_obj(so_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               nso_p_key, so_p_key, red_fc_p_key, red_em_p_key,
                               nso_m_key, so_m_key, red_fc_m_key, red_em_p_key,
                               nso_c_key, so_c_key, red_fc_c_key, red_em_p_key])
    
    
    for instance_name in nso_data:
        perfect_obj_nso = nso_data[instance_name][perfect_key]
        perfect_obj_so = so_data[instance_name][perfect_key]
        perfect_fc_red = abs(perfect_obj_so - perfect_obj_nso) / perfect_obj_nso
        
        mixed_obj_nso = nso_data[instance_name][mixed_key]
        mixed_obj_so = so_data[instance_name][mixed_key]
        mixed_fc_red = abs(mixed_obj_so - mixed_obj_nso) / mixed_obj_nso
        
        critical_obj_nso = nso_data[instance_name][critical_key]
        critical_obj_so = so_data[instance_name][critical_key]
        critical_fc_red = abs(critical_obj_so - critical_obj_nso) / critical_obj_nso
        
        row = pd.Series({instance_key: instance_name, 
                        nso_p_key: perfect_obj_nso, so_p_key: perfect_obj_so, red_fc_p_key: perfect_fc_red, 
                        nso_m_key: mixed_obj_nso, so_m_key: mixed_obj_so, red_fc_m_key: mixed_fc_red,
                        nso_c_key: critical_obj_nso, so_c_key: critical_obj_so, red_fc_c_key: critical_fc_red})
        
        df = df.append(row, ignore_index=True)
        
    return df
    
def map_instance_to_ws_to_obj(path):
    instance_to_ws_to_data = {}
    for file_name in os.listdir(path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        ws = split_name[1]
        seed = split_name[2]
        
        with open(path + file_name) as file:
            solution_json = json.load(file)
        
        obj = solution_json[best_obj_key]
        
        if instance_name in instance_to_ws_to_data:
            if ws in instance_to_ws_to_data[instance_name]:
                data = instance_to_ws_to_data[instance_name][ws]
                data[0] += obj
                data[1] += 1
            else:
                instance_to_ws_to_data[instance_name][ws] = [obj, 1]            
        else:
            instance_to_ws_to_data[instance_name] = {ws: [obj, 1]}
    
    instance_to_ws_to_obj = {}
    for instance_name in instance_to_ws_to_data:
        instance_to_ws_to_obj[instance_name] = {}
        for ws in instance_to_ws_to_data[instance_name]:
            data = instance_to_ws_to_data[instance_name][ws]
            instance_to_ws_to_obj[instance_name][ws] = data[0] / data[1]
            
    return instance_to_ws_to_obj

## Dataframe

In [76]:
df = generate_speed_opt_df(nso_path, so_path)
df

,instance,nso (p),so (p),cost red (p),em red (p),nso (m),so (m),cost red (m),em red (p),nso (c),so (c),cost red (c),em red (p)
0,19-21-3-2,6541.752,4971.400486,0.240051,NaN,7664.4165,6062.907685,0.208954,NaN,5898.5823,4405.181705,0.25318,NaN
